In [4]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable

import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict

# from graphsage.encoders import Encoder
# from graphsage.aggregators import MeanAggregator

from encoders import Encoder
from aggregators import MeanAggregator

"""
Simple supervised GraphSAGE model as well as examples running the model
on the Cora and Pubmed datasets.
"""

'\nSimple supervised GraphSAGE model as well as examples running the model\non the Cora and Pubmed datasets.\n'

In [5]:
class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        # 这里面赋值为enc2(经过两层GNN)
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()
        # 全连接参数矩阵，映射到labels num_classes维度做分类
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        # embeds实际是我们两层GNN后的输出nodes embedding
        embeds = self.enc(nodes)
        # 最后将nodes * hidden size 映射到 nodes * num_classes(= 7)之后做softmax计算cross entropy
        scores = self.weight.mm(embeds)
        return scores.t()

    def loss(self, nodes, labels):
        # 钱箱传播
        scores = self.forward(nodes)
        # 定义的cross entropy
        return self.xent(scores, labels.squeeze())

In [24]:
def load_cora():
    # 点的数量
    num_nodes = 2708
    # 特征数量
    num_feats = 1433
    # 构建特征矩阵
    feat_data = np.zeros((num_nodes, num_feats))
    # 构建节点的ground truth标签
    labels = np.empty((num_nodes,1), dtype=np.int64)
    # 做一个点的id映射
    node_map = {}
    label_map = {}

    # 读节点特征
    # cora.content第一列是node id, 中间为点的特征，最后一列为label
    # with open("cora/cora.content") as fp:
    with open("../cora/cora.content") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            # 特征，全部转换成float类型
            # feat_data[i,:] = map(float, info[1:-1])
            tmp = []
            for ss in info[1:-1]:
                tmp.append(float(ss))
            feat_data[i,:] = tmp
            
            # 将点的id转换，映射到从0开始的。info[0]是node old id,
            node_map[info[0]] = i
            # info[-1]是label, 字符串, 比如'Neural_Networks'和'Rule_Learning', 转换成int来表示类
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]

    # 读图存储成邻接表
    adj_lists = defaultdict(set)
    with open("../cora/cora.cites") as fp:
        for i,line in enumerate(fp):
            # 每一行是一条边
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    # 举例：(a, b) (a, c) (a, d) (b, c) (b, d)
    # 存储后 {a: set(b, c, d), b: set(a, c, d), c: set(a, b), d: set(a, b)}
    return feat_data, labels, adj_lists

In [28]:
def run_cora():
    # 随机数设置seed(种子)
    np.random.seed(1)
    random.seed(1)
    # cora数据集点数
    num_nodes = 2708
    # 加载cora数据集, 分别是
    # feat_data: 特征
    # labels: 标签
    # adj_lists: 邻接表，dict (key: node, value: neighbors set)
    feat_data, labels, adj_lists = load_cora()
    # 设置输入的input features矩阵X的维度 = 点的数量 * 特征维度
    features = nn.Embedding(2708, 1433)
    # 为矩阵X赋值，参数不更新
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
    # features.cuda()

    # 一共两层GNN layer
    # 第一层GNN
    # 以mean的方式聚合邻居, algorithm 1 line 4
    agg1 = MeanAggregator(features, cuda=True)
    # 将自身和聚合邻居的向量拼接后送入到神经网络(可选是否只用聚合邻居的信息来表示), algorithm 1 line 5
    enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)

    # 第二层GNN
    # 将第一层的GNN输出作为输入传进去
    # 这里面.t()表示转置，是因为Encoder class的输出维度为embed_dim * nodes
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    # enc1.embed_dim = 128, 变换后的维度还是128
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)

    # 采样的邻居点的数量
    enc1.num_samples = 5
    enc2.num_samples = 5

    # 7分类问题
    # enc2是经过两层GNN layer时候得到的 node embedding/features
    graphsage = SupervisedGraphSage(7, enc2)
    # graphsage.cuda()

    # 目的是打乱节点顺序
    rand_indices = np.random.permutation(num_nodes)

    # 划分测试集、验证集、训练集
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    # 用SGD的优化，设置学习率
    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    # 记录每个batch训练时间
    times = []
    # 共训练100个batch
    for batch in range(100):
        # 取256个nodes作为一个batch
        batch_nodes = train[:256]
        # 打乱训练集的顺序，使下次迭代batch随机
        random.shuffle(train)
        # 记录开始时间
        start_time = time.time()
        optimizer.zero_grad()
        # 这个是SupervisedGraphSage里面定义的cross entropy loss
        loss = graphsage.loss(batch_nodes, 
                Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        # 反向传播和更新参数
        loss.backward()
        optimizer.step()
        # 记录结束时间
        end_time = time.time()
        times.append(end_time-start_time)
        # print (batch, loss.data[0])
        print (batch, loss.data)

    # 做validation
    val_output = graphsage.forward(val)
    # 计算micro F1 score
    print ("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    # 计算每个batch的平均训练时间
    print ("Average batch time:", np.mean(times))

In [29]:
def load_pubmed():
    #hardcoded for simplicity...
    num_nodes = 19717
    num_feats = 500
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes, 1), dtype=np.int64)
    node_map = {}
    with open("pubmed-data/Pubmed-Diabetes.NODE.paper.tab") as fp:
        fp.readline()
        feat_map = {entry.split(":")[1]:i-1 for i,entry in enumerate(fp.readline().split("\t"))}
        for i, line in enumerate(fp):
            info = line.split("\t")
            node_map[info[0]] = i
            labels[i] = int(info[1].split("=")[1])-1
            for word_info in info[2:-1]:
                word_info = word_info.split("=")
                feat_data[i][feat_map[word_info[0]]] = float(word_info[1])
    adj_lists = defaultdict(set)
    with open("pubmed-data/Pubmed-Diabetes.DIRECTED.cites.tab") as fp:
        fp.readline()
        fp.readline()
        for line in fp:
            info = line.strip().split("\t")
            paper1 = node_map[info[1].split(":")[1]]
            paper2 = node_map[info[-1].split(":")[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists

In [30]:
def run_pubmed():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 19717
    feat_data, labels, adj_lists = load_pubmed()
    features = nn.Embedding(19717, 500)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
   # features.cuda()

    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 500, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 10
    enc2.num_samples = 25

    graphsage = SupervisedGraphSage(3, enc2)
#    graphsage.cuda()
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    times = []
    for batch in range(200):
        batch_nodes = train[:1024]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, 
                Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time-start_time)
        print (batch, loss.data[0])

    val_output = graphsage.forward(val) 
    print ("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print ("Average batch time:", np.mean(times))

In [31]:
if __name__ == "__main__":
    run_cora()

/opt/anaconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  # Remove the CWD from sys.path while we load stuff.


0 tensor(1.9526)
1 tensor(1.9337)
2 tensor(1.9120)
3 tensor(1.8948)
4 tensor(1.8646)
5 tensor(1.8379)
6 tensor(1.8061)
7 tensor(1.7765)
8 tensor(1.7303)
9 tensor(1.6885)
10 tensor(1.6025)
11 tensor(1.5917)
12 tensor(1.5437)
13 tensor(1.5093)
14 tensor(1.4577)
15 tensor(1.3794)
16 tensor(1.3457)
17 tensor(1.2853)
18 tensor(1.1386)
19 tensor(1.0898)
20 tensor(1.0590)
21 tensor(1.0387)
22 tensor(0.9482)
23 tensor(0.8871)
24 tensor(0.8640)
25 tensor(0.8847)
26 tensor(0.7707)
27 tensor(0.7469)
28 tensor(0.7268)
29 tensor(0.6874)
30 tensor(0.6186)
31 tensor(0.6889)
32 tensor(0.6337)
33 tensor(0.6179)
34 tensor(0.6229)
35 tensor(0.6171)
36 tensor(0.6988)
37 tensor(0.5377)
38 tensor(0.4776)
39 tensor(0.5085)
40 tensor(0.5046)
41 tensor(0.4095)
42 tensor(0.3825)
43 tensor(0.4307)
44 tensor(0.4305)
45 tensor(0.3681)
46 tensor(0.3415)
47 tensor(0.3196)
48 tensor(0.3558)
49 tensor(0.4070)
50 tensor(0.3418)
51 tensor(0.4133)
52 tensor(0.4054)
53 tensor(0.4708)
54 tensor(0.4079)
55 tensor(0.5492)
56